In [10]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [16]:
## implementar a função que identifica a bandeira
## a função deve receber uma imagem e retornar a bandeira identificada
## a função deve retornar uma lista de tuplas, onde cada tupla contém as coordenadas de um dos cantos da bandeira
## [(PAIS, (x1, y2), (x2, y2)`),(PAIS, (x1, y2), (x2, y2)`),(PAIS, (x1, y2), (x2, y2)`),...]

def get_dominant_colors(img, section):

    hsv = cv2.cvtColor(section, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
    dominant = np.unravel_index(hist.argmax(), hist.shape)
    return dominant

def identifica_bandeira(img):

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    results = []
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 50 and h > 50:
            roi = img[y:y+h, x:x+w]
            
            x1, y1 = x, y
            x2, y2 = x + w, y + h
            
            height, width = roi.shape[:2]
            left = roi[:, :width//2]
            right = roi[:, width//2:]
            top = roi[:height//2, :]
            bottom = roi[height//2:, :]
 
            red_hsv = (0, 200, 200)  
            white_hsv = (0, 0, 255) 
            green_hsv = (60, 200, 200) 
            orange_hsv = (30, 200, 200) 
            
            country = "unknown"
            
            if np.allclose(get_dominant_colors(left), red_hsv, atol=30) and \
               np.allclose(get_dominant_colors(right), white_hsv, atol=30):
                country = "monaco"
                
            elif np.allclose(get_dominant_colors(left), red_hsv, atol=30) and \
                 np.allclose(get_dominant_colors(right), red_hsv, atol=30):
                center = roi[:, width//4:3*width//4]
                if np.allclose(get_dominant_colors(center), white_hsv, atol=30):
                    country = "peru"
                    
            elif np.allclose(get_dominant_colors(top), red_hsv, atol=30) and \
                 np.allclose(get_dominant_colors(bottom), white_hsv, atol=30):
                country = "singapura"
                
            elif np.allclose(get_dominant_colors(left), green_hsv, atol=30) and \
                 np.allclose(get_dominant_colors(right), orange_hsv, atol=30):
                center = roi[:, width//3:2*width//3]
                if np.allclose(get_dominant_colors(center), white_hsv, atol=30):
                    country = "irlanda"
                    
            elif np.allclose(get_dominant_colors(left), green_hsv, atol=30) and \
                 np.allclose(get_dominant_colors(right), red_hsv, atol=30):
                center = roi[:, width//3:2*width//3]
                if np.allclose(get_dominant_colors(center), white_hsv, atol=30):
                    country = "italia"
            
            if country != "unknown":
                results.append((country, (x1, y1), (x2, y2)))
    
    return results

def draw_bandeiras(img, bandeiras):
    img_copy = img.copy()
    for (pais, (x1, y1), (x2, y2)) in bandeiras:
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_copy, pais, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return img_copy



In [33]:
## nao alterar essa função
# Função para desenhar as bandeiras identificadas
def draw_bandeiras(lista_bandeiras, bgr):
    try:
        for bandeira in lista_bandeiras:
            cv2.rectangle(bgr, bandeira[1], bandeira[2], (255, 0, 0), 5)
            cv2.putText(bgr, bandeira[0], bandeira[1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        return bgr
    except Exception as e:
        return bgr

In [ ]:
## Para testar a função identifica_bandeira

# Você pode testar a função identifica_bandeira com as imagens de teste fornecidas
# 
img = cv2.imread('img/teste1.png')
# img = cv2.imread('img/teste2.png')
# img = cv2.imread('img/teste3.png')
# img = cv2.imread('img/teste4.png')

# Chamar a função identifica_bandeira
resultado = identifica_bandeira(img)

# Imprimir o resultado da função identifica_bandeira 
print(resultado)

# Desenhar as bandeiras na imagem
bgr = draw_bandeiras(resultado, img)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB))
plt.show()


